In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle
from helper_scripts.ml_helpers import *
pd.set_option('display.max_columns', None)

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

mapping = {"0": "TotalTimeStopped_p20", "1": "TotalTimeStopped_p50", "2": "TotalTimeStopped_p80", "3": "DistanceToFirstStop_p20", "4": "DistanceToFirstStop_p50", "5": "DistanceToFirstStop_p80"}

In [2]:
df = pd.read_csv('data/train.csv')
df['Same_Street'] = df['EntryStreetName']==df['ExitStreetName']
df = shuffle(df)
df.head()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,TotalTimeStopped_p20,TotalTimeStopped_p40,TotalTimeStopped_p50,TotalTimeStopped_p60,TotalTimeStopped_p80,TimeFromFirstStop_p20,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,Same_Street
357167,2277502,498,41.86610,-87.73007,West Roosevelt Road,West Roosevelt Road,W,W,14,0,9,West Roosevelt Road_W_West Roosevelt Road_W,0,0,8,12,18,0,0,20,23,27,0.0,0.0,61.2,71.9,103.0,Chicago,True
554471,2474806,416,39.95384,-75.20026,Walnut Street,Walnut Street,W,W,8,0,7,Walnut Street_W_Walnut Street_W,0,0,0,0,11,0,0,0,0,23,0.0,0.0,0.0,0.0,69.5,Philadelphia,True
644717,2565052,878,39.97497,-75.19492,North 34th Street,North 34th Street,N,NW,22,0,12,North 34th Street_N_North 34th Street_NW,0,0,0,0,8,0,0,0,0,14,0.0,0.0,0.0,0.0,21.7,Philadelphia,True
212891,2133226,369,42.36476,-71.11077,Western Avenue,Western Avenue,W,W,16,0,7,Western Avenue_W_Western Avenue_W,0,0,0,0,17,0,0,0,0,31,0.0,0.0,0.0,0.0,64.8,Boston,True
161309,2081644,42,42.34631,-71.04835,South Boston Bypass Road,South Boston Bypass Road,NE,NE,8,0,7,South Boston Bypass Road_NE_South Boston Bypas...,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,Boston,True


In [3]:
df_test = pd.read_csv('data/test.csv')
df_test['Same_Street'] = df_test['EntryStreetName']==df_test['ExitStreetName']
df_test.head()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,City,Same_Street
0,0,1,33.75094,-84.39303,Peachtree Street Southwest,Mitchell Street Southwest,SW,SE,0,0,6,Peachtree Street Southwest_SW_Mitchell Street ...,Atlanta,False
1,1,1,33.75094,-84.39303,Peachtree Street Southwest,Peachtree Street Southwest,SW,SW,0,0,6,Peachtree Street Southwest_SW_Peachtree Street...,Atlanta,True
2,2,1,33.75094,-84.39303,Peachtree Street Southwest,Peachtree Street Southwest,NE,NE,1,0,6,Peachtree Street Southwest_NE_Peachtree Street...,Atlanta,True
3,3,1,33.75094,-84.39303,Peachtree Street Southwest,Peachtree Street Southwest,SW,SW,1,0,6,Peachtree Street Southwest_SW_Peachtree Street...,Atlanta,True
4,4,1,33.75094,-84.39303,Peachtree Street Southwest,Peachtree Street Southwest,NE,NE,2,0,6,Peachtree Street Southwest_NE_Peachtree Street...,Atlanta,True


In [4]:
df[df['EntryStreetName']!=df['ExitStreetName']]

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,TotalTimeStopped_p20,TotalTimeStopped_p40,TotalTimeStopped_p50,TotalTimeStopped_p60,TotalTimeStopped_p80,TimeFromFirstStop_p20,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,Same_Street
812817,2733152,1767,39.98629,-75.15621,West Susquehanna Avenue,North Broad Street,W,S,19,0,6,West Susquehanna Avenue_W_North Broad Street_S,30,34,41,46,61,43,59,71,86,155,67.5,68.5,79.0,385.4,547.6,Philadelphia,False
197059,2117394,250,42.26162,-71.12217,Hyde Park Avenue,West Street,SW,W,14,0,8,Hyde Park Avenue_SW_West Street_W,0,0,0,7,40,0,0,0,27,56,0.0,0.0,0.0,45.8,113.9,Boston,False
142140,2062475,490,33.77258,-84.38026,Myrtle Street Northeast,Ponce de Leon Avenue Northeast,S,E,20,0,11,Myrtle Street Northeast_S_Ponce de Leon Avenue...,8,33,33,72,94,49,51,51,81,102,52.6,54.5,54.5,80.2,86.8,Atlanta,False
279668,2200003,828,42.33570,-71.08703,Tremont Street,Melnea Cass Boulevard,E,E,11,1,7,Tremont Street_E_Melnea Cass Boulevard_E,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,Boston,False
251788,2172123,637,42.29937,-71.07364,Washington Street,Bowdoin Street,SE,E,11,0,9,Washington Street_SE_Bowdoin Street_E,0,0,0,13,25,0,0,0,21,40,0.0,0.0,0.0,59.5,71.1,Boston,False
115774,2036109,382,33.74197,-84.39823,Fulton Street Southwest,Windsor Street Southwest,W,S,11,0,7,Fulton Street Southwest_W_Windsor Street South...,0,0,0,0,8,0,0,0,0,45,0.0,0.0,0.0,0.0,284.0,Atlanta,False
827836,2748171,1838,39.95279,-75.16684,South 16th Street,North 16th Street,NE,NE,14,1,11,South 16th Street_NE_North 16th Street_NE,0,0,0,13,17,0,0,0,19,31,0.0,0.0,0.0,34.9,84.1,Philadelphia,False
569368,2489703,480,39.98484,-75.23724,North 59th Street,Upland Way,NE,E,0,1,8,North 59th Street_NE_Upland Way_E,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,Philadelphia,False
805217,2725552,1724,39.95183,-75.18406,South 30th Street,Walnut Street,S,W,3,1,9,South 30th Street_S_Walnut Street_W,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,Philadelphia,False
528963,2449298,293,39.95600,-75.21764,South 48th Street,Walnut Street,S,W,11,0,11,South 48th Street_S_Walnut Street_W,0,0,0,0,31,0,0,0,0,37,0.0,0.0,0.0,0.0,50.4,Philadelphia,False


In [5]:
df[(df[[mapping[str(x)] for x in range(6)]] > 0).any(axis=1)]


,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,TotalTimeStopped_p20,TotalTimeStopped_p40,TotalTimeStopped_p50,TotalTimeStopped_p60,TotalTimeStopped_p80,TimeFromFirstStop_p20,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,Same_Street
357167,2277502,498,41.86610,-87.73007,West Roosevelt Road,West Roosevelt Road,W,W,14,0,9,West Roosevelt Road_W_West Roosevelt Road_W,0,0,8,12,18,0,0,20,23,27,0.0,0.0,61.2,71.9,103.0,Chicago,True
554471,2474806,416,39.95384,-75.20026,Walnut Street,Walnut Street,W,W,8,0,7,Walnut Street_W_Walnut Street_W,0,0,0,0,11,0,0,0,0,23,0.0,0.0,0.0,0.0,69.5,Philadelphia,True
644717,2565052,878,39.97497,-75.19492,North 34th Street,North 34th Street,N,NW,22,0,12,North 34th Street_N_North 34th Street_NW,0,0,0,0,8,0,0,0,0,14,0.0,0.0,0.0,0.0,21.7,Philadelphia,True
212891,2133226,369,42.36476,-71.11077,Western Avenue,Western Avenue,W,W,16,0,7,Western Avenue_W_Western Avenue_W,0,0,0,0,17,0,0,0,0,31,0.0,0.0,0.0,0.0,64.8,Boston,True
481081,2401416,45,39.95564,-75.20227,South 40th Street,South 40th Street,S,S,19,0,8,South 40th Street_S_South 40th Street_S,0,12,22,29,46,0,35,38,44,58,0.0,71.1,73.7,76.1,99.7,Philadelphia,True
851574,2771909,1975,39.96732,-75.24579,Vine Street,Vine Street,E,E,12,0,8,Vine Street_E_Vine Street_E,0,0,6,9,14,0,0,13,20,36,0.0,0.0,56.1,59.1,73.8,Philadelphia,True
709336,2629671,1193,39.93292,-75.14872,East Moyamensing Avenue,East Moyamensing Avenue,SW,SW,12,0,11,East Moyamensing Avenue_SW_East Moyamensing Av...,0,6,6,12,12,0,12,12,17,20,0.0,47.8,47.8,52.6,56.2,Philadelphia,True
44619,1964954,168,33.77242,-84.38331,Ponce de Leon Avenue Northeast,Ponce de Leon Avenue Northeast,E,E,1,0,6,Ponce de Leon Avenue Northeast_E_Ponce de Leon...,0,8,25,32,47,0,12,34,36,53,0.0,51.5,61.3,61.4,67.1,Atlanta,True
266907,2187242,734,42.36015,-71.09488,Massachusetts Avenue,Massachusetts Avenue,NW,NW,20,1,10,Massachusetts Avenue_NW_Massachusetts Avenue_NW,0,7,17,27,42,0,16,30,45,50,0.0,61.2,62.7,68.0,105.8,Boston,True
736888,2657223,1349,39.93384,-75.14728,Washington Avenue,Washington Avenue,E,E,7,1,8,Washington Avenue_E_Washington Avenue_E,0,0,0,0,12,0,0,0,0,21,0.0,0.0,0.0,0.0,57.0,Philadelphia,True


In [6]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [10]:
feature_names = ['IntersectionId', 'EntryHeading', 'ExitHeading', 'Hour', 'Weekend', 'Month', 'Same_Street', 'Latitude', 'Longitude']
X_test = df_test[feature_names].copy()
X = df[feature_names].copy()
train_cats(X)
apply_cats(X_test, X)

cat_cols = [x for x in X.columns if X[x].dtype.name=='category']
for col in cat_cols:
    numericalize(X, X[col], col+'_num', None)
X.drop(cat_cols, axis=1, inplace=True)
    
cat_cols = [x for x in X_test.columns if X_test[x].dtype.name=='category']
for col in cat_cols:
    numericalize(X_test, X_test[col], col+'_num', None)
X_test.drop(cat_cols, axis=1, inplace=True)

n_valid = 100000  
n_trn = len(X)-n_valid
X_train, X_valid = split_vals(X, n_trn)

for x in range(6):
    print(mapping[str(x)])
    y = df[mapping[str(x)]]
    y_train, y_valid = split_vals(y, n_trn)
    m = RandomForestRegressor(n_estimators=40, n_jobs=-1, min_samples_leaf=4)
    %time m.fit(X_train, y_train)
    print_score(m)

#X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

TotalTimeStopped_p20
Wall time: 32.8 s
[4.429605435910728, 5.506614801790503, 0.6084999772536099, 0.3958032993700187]
TotalTimeStopped_p50
Wall time: 32.4 s
[7.564201316171499, 9.637137320558399, 0.7634236186265214, 0.6167186022866531]
TotalTimeStopped_p80
Wall time: 33.2 s
[12.906425985678448, 16.289968195932193, 0.7894983270604128, 0.6621007781418904]
DistanceToFirstStop_p20
Wall time: 30.3 s
[18.587461363815095, 22.80942320115656, 0.5609070014235751, 0.31908161619245157]
DistanceToFirstStop_p50
Wall time: 35.4 s
[38.4289962511719, 48.877809590876645, 0.7129591344217308, 0.5347894876937127]
DistanceToFirstStop_p80
Wall time: 38 s
[62.94396262508745, 82.92857850255662, 0.82981694463231, 0.7079860064962109]


In [11]:
with open('data/submissions.csv', 'w') as f:
    pd.DataFrame(columns=['TargetId', 'Target']).to_csv(f, index=False)

for x in range(6):
    y = df[mapping[str(x)]]
    m = RandomForestRegressor(n_estimators=40, n_jobs=-1, min_samples_leaf=4)
    %time m.fit(X, y)
    print(rmse(m.predict(X_train), y_train), m.score(X,y))
    df_submission = pd.concat([(df_test['RowId'].astype(str) + f'_{x}'), pd.Series(m.predict(X_test))],axis=1)
    with open('data/submissions.csv', 'a') as f:
        df_submission.to_csv(f, index=False, header=None)

Wall time: 39.9 s
171.0444039593109 0.6141027546703011
Wall time: 38.1 s
165.57221446764683 0.7684166390781088
Wall time: 38.4 s
155.66529027890311 0.7934233742684911
Wall time: 34.5 s
163.03557154503264 0.5630997078841067
Wall time: 36.1 s
129.60826693527412 0.7139313434813023
Wall time: 40.9 s
62.74350914615876 0.8299423654126812
